# Create virtual machines in Orbstack

This setup is meant for running Ansible playbooks on MacOS. 
For Linux support, set up two virtual machines with the the tool of your preference, boot up the number of machines you wish to provision, [create required users](https://documentation.opencrvs.org/setup/3.-installation/3.3-set-up-a-server-hosted-environment/3.3.1-provision-your-server-nodes-with-ssh-access#create-a-user-named-provision) and create a variation of `local.macos.yml` inventory file. 

## Prerequisites
1. MacOS
2. Install [OrbStack](orbstack.dev)
3. Confirm `orb` command is now available on your command line

Either run the commands manually from command like (for live output) or by executing cells here

In [ ]:
%%bash
bash ./provision-macos.sh

Notice that the following task will prompt (in VSCode) for your dockerhub username and password

In [ ]:
import os

os.environ["DOCKER_USERNAME"] = input("Enter Dockerhub username: ")
os.environ["DOCKER_PASSWORD"] = input("Enter Dockerhub password: ")

The next command is run with Python instead of %%bash simply because %%bash doesn't support real-time output

In [ ]:
cmd = """
mongo_user=$(openssl rand -hex 8)
mongo_pass=$(openssl rand -base64 24)
backup_pass=$(openssl rand -base64 16)
es_pass=$(openssl rand -base64 24)

ansible-playbook -i local.macos.yml ../server-setup/playbook.yml \\
  -e encrypted_disk_size=200g \\
  -e dockerhub_username=$DOCKER_USERNAME \\
  -e dockerhub_password=$DOCKER_PASSWORD \\
  -e mongodb_admin_username=$mongo_user \\
  -e mongodb_admin_password="$mongo_pass" \\
  -e backup_encryption_passphrase="$backup_pass" \\
  -e elasticsearch_superuser_password="$es_pass"
"""

In [ ]:
import subprocess

proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, shell=True, env=os.environ)
for line in proc.stdout:
    print(line, end="")
proc.wait()